In [2]:
# Output saving config
OUTPUT_DIR = 'outputs'
from utils import ensure_dir
ensure_dir(OUTPUT_DIR)


'outputs'

# Experiment 5: Prompt Ablation Study

Evaluate impact of increasingly constrained prompts on validity & action quality.

In [3]:
MODEL_KEY = 'qwen25'
PROMPTS = {
  'minimal': 'What should the robot do?',
  'structured': 'You are a driving assistant. Output ONLY JSON {"action": "FORWARD|LEFT|RIGHT|STOP|SLOW", "rationale": "<short reason>"}.',
  'rich': 'Fuse perception. Output ONLY JSON with action rationale referencing lane & obstacles: {"action":...,"rationale":...}'
}

In [4]:
from PIL import Image
from utils import list_frame_pairs, generate_action, parse_action_json
pairs = list_frame_pairs(folder_name='example_edge_samples')
len(pairs)

6

In [5]:
results = {}
all_outputs=[]
for label,prompt in PROMPTS.items():
    parsed = []
    for cam_path,_ in pairs:
        img = Image.open(cam_path).convert('RGB')
        out = generate_action(MODEL_KEY, [img], prompt)
        pj = parse_action_json(out) or {}
        parsed.append(pj)
        all_outputs.append(out)
    validity = sum(1 for p in parsed if 'action' in p)/len(parsed)
    results[label] = validity
results

/home/o0i3z3/thesis/vlm_laboratories/vlm_laboratories/newvens/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

{'minimal': 0.0, 'structured': 1.0, 'rich': 0.5}

In [6]:
# Persist prompt validity results
if 'results' in globals():
    import json, os
    from utils import save_text
    save_text(os.path.join(OUTPUT_DIR,'experiment5_prompt_ablation_summary.json'), json.dumps(results, indent=2))
    results
else:
    print('Run ablation cell first.')